# Scraping usedguns.com

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import altair as alt
import numpy as np
import datetime as dt
import re
import time
import random
from tqdm import tqdm

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

In [5]:
url = "https://www.usedguns.com/Subcategory/Pistols/8423042"

In [6]:
headers = {
    "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="101", "Google Chrome";v="101"',
    "Referer": "https://www.gunviolencearchive.org/",
    "sec-ch-ua-mobile": "?0",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.0.0 Safari/537.36",
    "sec-ch-ua-platform": '"macOS"',
}

#### Construct the urls for each category and page number

In [7]:
formaturl = (
    lambda x: "https://www.usedguns.com/Subcategory/"
    + f"{x[0]}"
    + "?page="
    + f"{x[1]}"
    + "&pageSize=100&sort=7&listView=true"
)

#### Increase the range to get more pages

In [8]:
metadata = []
urls = []
for p in range(1, 50):
    for c in ["Rifles/33588866", "Pistols/8423042"]:
        # metadata.append(dict(page=p, category=c, url=formaturl((c, p))))
        urls.append(formaturl((c, p)))

#### Slowly loop through the urls and snag items from each gun listing

In [10]:
dicts = []

for u in tqdm(urls):
    response = requests.get(u, headers=headers)
    soup = BeautifulSoup(response.text)
    articles = soup.find_all("article")

    for d in articles:
        for item in d.find_all("article"):
            data_dict = {
                "short_title": item.find("h3").text,
                "quality": item.find("p", class_="quality").text,
                "price": re.sub(
                    r"[\n\t\s]*",
                    "",
                    item.find("span", class_="price-without-icon").text,
                ),
                "description": item.find("p", class_="description").text,
                "url": item.find("a", href=True)["href"],
                # "thumbnail": item.find("img", class_="model-thumbnail")["src"],
                # "category": category_url,
            }
            dicts.append(data_dict)

    time.sleep(random.randint(3, 6))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 98/98 [11:24<00:00,  6.99s/it]


#### Convert the list of dictionaries into a dataframe

In [11]:
src_df = pd.DataFrame(dicts)

In [12]:
src_df = src_df.drop_duplicates()

#### How many guns?

In [13]:
count_guns = len(src_df)
count_guns

5818

#### Categorize by firearm type

In [14]:
src_df.loc[src_df["description"].str.lower().str.contains("pistol"), "type"] = "pistol"
src_df.loc[src_df["description"].str.lower().str.contains("rifle"), "type"] = "rifle"

In [15]:
src_df.head()

,short_title,quality,price,description,url,type
0,"Anderson AM15 5.56mm 16"" Semi-Auto Rifle - Black",Good,$599.99,"RIFLE: ANDERSON MANUFACTURING MODEL AM-15 RIFLE, MATTE BLACK, NOT INCLUDED, SEMI - AUTOMATIC, 5.56MM, 16.5",/Item/Details/Anderson-AM15-5-56mm-16-Semi-Auto-Rifle-Black/3175a286c37e44e4b59405038416ae72,rifle
1,ANDERSON MANUFACTURING AM-15 RIFLE,Like New,$524.95,"RIFLE: ANDERSON MANUFACTURING MODEL AM-15 RIFLE, OTHER, NOT INCLUDED, 16"", 5.56MM, SEMI - AUTOMATIC",/Item/Details/ANDERSON-MANUFACTURING-AM-15-RIFLE/f4ddde6787184d0ab3bc33eabe04f9ed,rifle
2,ANDERSON MANUFACTURING AM-15 RIFLE,Like New,$679.99,"RIFLE: ANDERSON MANUFACTURING MODEL AM-15 RIFLE, BLUE, NOT INCLUDED, 20IN, 5.56MM, SEMI - AUTOMATIC, W/ AERO PRECISION 20IN BULL BARREL UPPER",/Item/Details/ANDERSON-MANUFACTURING-AM-15-RIFLE/b4254fa630b4473ca39985c70071f178,rifle
3,ANDERSON MANUFACTURING AM-15 RIFLE,Very Good,$699.99,"RIFLE: ANDERSON MANUFACTURING MODEL AM-15 RIFLE, MATTE BLACK, NOT INCLUDED, 16, 5.56MM, SEMI - AUTOMATIC",/Item/Details/ANDERSON-MANUFACTURING-AM-15-RIFLE/5ecf98606c8a4f13929e3896ee9797a6,rifle
4,ANDERSON MANUFACTURING AM-15 RIFLE,Brand New,$799.99,"RIFLE: ANDERSON MANUFACTURING MODEL AM-15 RIFLE, MATTE BLACK, 16, 5.56 NATO 1:8, SEMI - AUTOMATIC",/Item/Details/ANDERSON-MANUFACTURING-AM-15-RIFLE/8975aa78c6734f779542c4fc615e81ae,rifle


---

## Export

In [16]:
src_df.to_csv(
    f"data/processed/usedgunsdotcom_sample_{count_guns}_scraped_pistols_rifles_{month_year_updated}.csv",
    index=False,
)